In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
import torch
from transformers import pipeline
import keyboard

# Ses kaydı parametreleri
SAMPLE_RATE = 44100  # 44.1 kHz

def record_audio():
    print("Ses kaydediliyor... Enter'a tekrar basarak kaydı bitirebilirsiniz.")
    
    recording = []
    def callback(indata, frames, time, status):
        if status:
            print(status)
        recording.append(indata.copy())
    
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype=np.int16, callback=callback):
        keyboard.wait("enter")  # Enter'a basılınca kayıt bitirilecek
    
    audio_data = np.concatenate(recording, axis=0)
    wav.write("recorded_audio.wav", SAMPLE_RATE, audio_data)
    print("Ses kaydı tamamlandı!")
    return "recorded_audio.wav"

def transcribe_audio(audio_file):
    print("Ses metne dönüştürülüyor...")
    model = whisper.load_model("base")
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)
    
    # Konuşulan dili tespit et
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    print("Transkripsiyon:", result.text)
    return result.text

def analyze_sentiment(text):
    print("Duygu analizi yapılıyor...")
    sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
    result = sentiment_pipeline(text)
    print("Duygu Analizi Sonucu:", result)
    return result

def main():
    input("Ses kaydetmek için Enter'a basın...")
    audio_file = record_audio()
    text = transcribe_audio(audio_file)
    sentiment = analyze_sentiment(text)
    print("Sonuç:", sentiment)

if __name__ == "__main__":
    main()